In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# !pip install wandb -qqq
import wandb
# wandb.login()
api = wandb.Api()
import os

os.environ["WANDB_API_KEY"] = "0f780ac8a470afe6cb7fc474ff3794772c660465"

os.environ["WANDB_NOTEBOOK_NAME"] = "jup_res"


# selected_tag = 'Pred'

runs = api.runs("hokarami/TEEDAM_supervised")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/hokarami/.netrc


In [3]:
import pandas as pd 


# Project is specified by <entity/project-name>
def dl_runs(all_runs, selected_tag=None):




    summary_list, config_list, name_list = [], [], []
    for run in runs: 

        if (selected_tag not in run.tags) and (selected_tag is not None):
            continue
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        

        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list
        })

    # runs_df.to_csv("project.csv")
    return runs_df

In [4]:
# q = runs_df['name'].str.contains('\[R2-')
# runs_df = runs_df[q]


runs_df = dl_runs(runs, selected_tag='H20')

runs_df
runs_df.iloc[0].summary.keys()
runs_df.iloc[0].config.keys()

,summary,config,name
0,"{'Obj': 0.4786324786324786, '_timestamp': 1677...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-DA__base-concat]1664731
1,"{'_step': 26, '_timestamp': 1677774787.7543075...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-TEDA__nextmark-concat]1667036
2,{'Test-NextType(ML)/f1-weighted': 0.0032353428...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-DA__base-concat]1663219
3,{'Best-Valid-pred_label/AUROC': 0.880521512385...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-TEDA__nextmark-concat]1664506
4,"{'Valid-pred_label/AUROC': 0.846805736636245, ...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-DA__base-concat]1666267
...,...,...,...
103,"{'Test-NextTime/RMSE': 2.5213950694430203, 'Te...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-DA__base-concat]1644008
104,"{'_runtime': 3486.1978652477264, '_timestamp':...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-TEDA__nextmark-concat]1649688
105,{'Best-Valid-NextType(ML)/f1-weighted': 0.0469...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-DA__base-concat]1644494
106,{'Best-Valid-pred_label/AUPRC': 0.573192315423...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-TEDA__nextmark-concat]1646207


dict_keys(['Obj', '_timestamp', 'Test-NextTime/RMSE', 'Test-pred_label/loss', 'Best-Valid-pred_label/AUROC', 'Test-NextType(ML)/f1-weighted', 'Valid-pred_label/recall-binary', 'Valid-pred_label/AUPRC', 'Best-Test-pred_label/AUPRC', 'Best-Test-pred_label/AUROC', 'Valid-pred_label/f1-binary', '_runtime', 'Test-NextTime/msae', 'Valid-NextTime/RMSE', 'Valid-pred_label/loss', 'time-Time/valid_epoch', 'Valid-NextType(ML)/f1-weighted', 'Best-Valid-pred_label/f1-binary', 'Valid-pred_label/precision-binary', 'Best-Test-NextType(ML)/f1-weighted', 'Valid-NextType(ML)/auc-ovo-weighted', 'Test-pred_label/AUROC', 'time-Time/train_epoch', 'Valid-pred_label/AUROC', 'Best-Valid-pred_label/AUPRC', 'Valid-NextType(ML)/recall-weighted', 'Best-Test-NextType(ML)/auc-ovo-weighted', 'Best-Valid-NextType(ML)/auc-ovo-weighted', '_step', 'Test-pred_label/AUPRC', 'Test-NextTime/rmse_norm', 'Valid-NextType(ML)/precision-weighted', 'Valid-NextTime/rmse_norm', 'Test-pred_label/precision-binary', 'Test-NextType(ML)/a

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'INPUT', 'epoch', 'noise', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'OUTPUT', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'log_freq', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'NOISE_config', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'wandb_project', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'dict_map_events', 'dict_map_states', 'next_time_config', 'next_type_config'

# Table I
Effect of time concatenation and single+mark

In [5]:
df_filt = dl_runs(runs, selected_tag='H20')
df_filt


,summary,config,name
0,"{'Obj': 0.4786324786324786, '_timestamp': 1677...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-DA__base-concat]1664731
1,"{'_step': 26, '_timestamp': 1677774787.7543075...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-TEDA__nextmark-concat]1667036
2,{'Test-NextType(ML)/f1-weighted': 0.0032353428...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-DA__base-concat]1663219
3,{'Best-Valid-pred_label/AUROC': 0.880521512385...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-TEDA__nextmark-concat]1664506
4,"{'Valid-pred_label/AUROC': 0.846805736636245, ...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-DA__base-concat]1666267
...,...,...,...
103,"{'Test-NextTime/RMSE': 2.5213950694430203, 'Te...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-DA__base-concat]1644008
104,"{'_runtime': 3486.1978652477264, '_timestamp':...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-TEDA__nextmark-concat]1649688
105,{'Best-Valid-NextType(ML)/f1-weighted': 0.0469...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-DA__base-concat]1644494
106,{'Best-Valid-pred_label/AUPRC': 0.573192315423...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H10-TEDA__nextmark-concat]1646207


In [6]:
i_groupby_row =['dataset']
i_groupby_col = ['time_enc']
i_agg = ['split']
i_metric = ['Best-Test-CIF/LL-#events', 'Test-NextType(ML)/auc-ovo-weighted','Test-NextType(ML)/f1-weighted']
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]


# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),2))  + ' (' + str(round(x.std(),2))  +')'  ).reset_index() #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1


df_result = a1.melt(id_vars=i_groupby_row + i_groupby_col,value_vars=i_metric, var_name='Metric')
# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')

df_result.pivot(index=i_groupby_row+['Metric'],columns=i_groupby_col)

# a1.reset_index().pivot(index=i_groupby_row+i_metric,columns=i_groupby_col)



# a1.to_excel("aaa.xlsx")

KeyError: "['Best-Test-CIF/LL-#events'] not in index"

# Table III

In [7]:
df_filt = dl_runs(runs, selected_tag='H20')

In [9]:
i_groupby_row =['dataset','setting','test_center']
i_groupby_col = ['event_enc','mod']
i_groupby_col=['OUTPUT','INPUT']
i_agg = ['split']
i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC','Best-Test-pred_label/AUROC']
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),2))  + ' (' + str(round(x.std(),2))  +')'  ) #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_excel("aaa.xlsx")

,dataset,setting,test_center,OUTPUT,INPUT,split,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC,Best-Test-pred_label/AUROC
0,P12,mc1,2,none-mark-label,DAM,4,0.478632,0.515578,0.828240
1,P12,mc1,2,none-mark-label,TEDAM,3,0.530035,0.584094,0.859742
2,P12,mc1,2,none-mark-label,DAM,3,0.560311,0.606985,0.866970
3,P12,mc1,2,none-mark-label,TEDAM,2,0.591760,0.609171,0.880522
4,P12,mc1,2,none-mark-label,DAM,2,0.579832,0.633018,0.875424
...,...,...,...,...,...,...,...,...,...
103,P19,mc2,1,none-mark-label,DAM,,0.615034,0.527253,0.921468
104,P19,mc2,0,none-mark-label,TEDAM,,0.591313,0.668725,0.866591
105,P19,mc2,0,none-mark-label,DAM,,0.445486,0.676422,0.877705
106,P12,seft,,none-mark-label,TEDAM,,0.508321,0.508736,0.847576


Best-Test-pred_label/f1-binary  \
dataset setting test_center OUTPUT          INPUT                                  
P12     mc1     0           none-mark-label DAM                      0.53 (0.04)   
                                            TEDAM                    0.52 (0.03)   
                1           none-mark-label DAM                      0.56 (0.02)   
                                            TEDAM                    0.55 (0.03)   
                2           none-mark-label DAM                      0.55 (0.04)   
                                            TEDAM                    0.57 (0.03)   
        mc2     0           none-mark-label DAM                       0.52 (nan)   
                                            TEDAM                     0.52 (nan)   
                1           none-mark-label DAM                       0.54 (nan)   
                                            TEDAM                     0.54 (nan)   
                2           none-mark-label DAM                       0.52 (nan)   
                                            TEDAM                     0.54 (nan)   
        sc      0           none-mark-label DAM                      0.49 (0.05)   
                                            TEDAM                    0.46 (0.01)   
                1           none-mark-label DAM                      0.52 (0.03)   
                                            TEDAM                     0.5 (0.03)   
                2           none-mark-label DAM                      0.53 (0.03)   
                                            TEDAM                    0.52 (0.04)   
        seft                none-mark-label DAM                       0.51 (nan)   
                                            TEDAM                     0.51 (nan)   
P19     mc1     0           none-mark-label DAM                       0.7 (0.02)   
                                            TEDAM                    0.69 (0.01)   
                1           none-mark-label DAM                      0.62 (0.04)   
                                            TEDAM                    0.62 (0.03)   
        mc2     0           none-mark-label DAM                       0.45 (nan)   
                                            TEDAM                     0.59 (nan)   
                1           none-mark-label DAM                       0.62 (nan)   
                                            TEDAM                     0.61 (nan)   
        sc      0           none-mark-label DAM                      0.68 (0.02)   
                                            TEDAM                    0.68 (0.03)   
                1           none-mark-label DAM                      0.61 (0.02)   
                                            TEDAM                    0.61 (0.01)   

                                                  Best-Test-pred_label/AUPRC  \
dataset setting test_center OUTPUT          INPUT                              
P12     mc1     0           none-mark-label DAM                  0.53 (0.06)   
                                            TEDAM                0.53 (0.04)   
                1           none-mark-label DAM                  0.56 (0.05)   
                                            TEDAM                0.56 (0.03)   
                2           none-mark-label DAM                  0.59 (0.05)   
                                            TEDAM                0.58 (0.03)   
        mc2     0           none-mark-label DAM                   0.53 (nan)   
                                            TEDAM                 0.52 (nan)   
                1           none-mark-label DAM                   0.53 (nan)   
                                            TEDAM                 0.55 (nan)   
                2           none-mark-label DAM                   0.56 (nan)   
                                            TEDAM                 0.56 (nan)   
        sc      0           none-mark-label DAM                  0.48 (0.

Best-Test-pred_label/f1-binary               \
OUTPUT                                     none-mark-label                
INPUT                                                  DAM        TEDAM   
dataset setting test_center                                               
P12     mc1     0                              0.53 (0.04)  0.52 (0.03)   
                1                              0.56 (0.02)  0.55 (0.03)   
                2                              0.55 (0.04)  0.57 (0.03)   
        mc2     0                               0.52 (nan)   0.52 (nan)   
                1                               0.54 (nan)   0.54 (nan)   
                2                               0.52 (nan)   0.54 (nan)   
        sc      0                              0.49 (0.05)  0.46 (0.01)   
                1                              0.52 (0.03)   0.5 (0.03)   
                2                              0.53 (0.03)  0.52 (0.04)   
        seft                                    0.51 (nan)   0.51 (nan)   
P19     mc1     0                               0.7 (0.02)  0.69 (0.01)   
                1                              0.62 (0.04)  0.62 (0.03)   
        mc2     0                               0.45 (nan)   0.59 (nan)   
                1                               0.62 (nan)   0.61 (nan)   
        sc      0                              0.68 (0.02)  0.68 (0.03)   
                1                              0.61 (0.02)  0.61 (0.01)   

                            Best-Test-pred_label/AUPRC               \
OUTPUT                                 none-mark-label                
INPUT                                              DAM        TEDAM   
dataset setting test_center                                           
P12     mc1     0                          0.53 (0.06)  0.53 (0.04)   
                1                          0.56 (0.05)  0.56 (0.03)   
                2                          0.59 (0.05)  0.58 (0.03)   
        mc2     0                           0.53 (nan)   0.52 (nan)   
                1                           0.53 (nan)   0.55 (nan)   
                2                           0.56 (nan)   0.56 (nan)   
        sc      0                          0.48 (0.05)  0.44 (0.02)   
                1                          0.51 (0.05)  0.47 (0.03)   
                2                          0.51 (0.05)  0.53 (0.05)   
        seft                                0.51 (nan)   0.51 (nan)   
P19     mc1     0                          0.78 (0.02)  0.78 (0.02)   
                1                           0.7 (0.03)  0.69 (0.03)   
        mc2     0                           0.68 (nan)   0.67 (nan)   
                1                           0.53 (nan)   0.55 (nan)   
        sc      0                          0.77 (0.02)  0.76 (0.03)   
                1                          0.68 (0.01)  0.67 (0.01)   

                            Best-Test-pred_label/AUROC               
OUTPUT                                 none-mark-label               
INPUT                                              DAM        TEDAM  
dataset setting test_center                                          
P12     mc1     0                          0.85 (0.03)  0.85 (0.03)  
                1                          0.86 (0.01)  0.87 (0.01)  
                2                          0.86 (0.02)  0.87 (0.01)  
        mc2     0                           0.85 (nan)   0.85 (nan)  
                1                           0.85 (nan)   0.86 (nan)  
                2                           0.85 (nan)   0.86 (nan)  
        sc      0                          0.82 (0.03)  0.81 (0.02)  
                1                          0.83 (0.01)  0.82 (0.01)  
                2                          0.84 (0.03)  0.84 (0.02)  
        seft                                0.85 (nan)   0.85 (nan)  
P19     mc1     0                          0.92 (0.02)  0.93 (0.01)  
                1                          0.94 (0.01)  

In [45]:
df

,dataset,setting,test_center,OUTPUT,INPUT,split,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC,Best-Test-pred_label/AUROC
0,/scratch/hokarami/data_tedam/p12/,mc2,2,none-label,TEDAM,{0},0.422311,0.378606,0.776002
1,/scratch/hokarami/data_tedam/p12/,mc2,2,none-label,DAM,{0},0.421284,0.407693,0.784718
2,/scratch/hokarami/data_tedam/p12/,mc2,1,none-label,TEDAM,{0},0.398410,0.357587,0.767543
3,/scratch/hokarami/data_tedam/p12/,mc2,1,none-label,DAM,{0},0.430396,0.407364,0.789033
4,/scratch/hokarami/data_tedam/p12/,mc2,0,none-label,TEDAM,{0},0.412155,0.404788,0.789505
...,...,...,...,...,...,...,...,...,...
169,/scratch/hokarami/data_tedam/p12/,mc1,0,single-mark,TE,1,NaN,NaN,NaN
170,/scratch/hokarami/data_tedam/p12/,mc1,0,none-label,TEDAM,0,0.569014,0.699770,0.945812
171,/scratch/hokarami/data_tedam/p12/,mc1,0,none-label,DAM,0,0.459658,0.479535,0.861373
172,/scratch/hokarami/data_tedam/p12/,mc1,0,single-mark,TEDAM,0,NaN,NaN,NaN


# CIF

In [7]:
df_filt = dl_runs(runs, selected_tag='CIF')

df_filt.iloc[0].config.keys()

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

In [43]:
i_groupby_row =['dataset','setting',]
i_groupby_col = ['state']
i_agg = ['test_center','split']
# i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC']
i_metric = ['Test-CIF/NLL-#events']


#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)#.dropna(axis=1,how='all')
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df





a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(['mean','std']).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),3))  + ' (' + str(round(x.std(),3))  +')' )

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_csv("aaa.csv")

KeyError: "['Test-CIF/NLL-#events'] not in index"